In [1]:
import pandas as pd
import datetime
import sqlite3
import time
import re
import calendar
import schedule

## Setting Up Database
- Download the database for project from [here](https://drive.google.com/file/d/1TD0mjsv-Vkz7JKGXCikGf65OYzeamiL2/view?usp=sharing)
- Check the table names in the database. There should be two tables:
    - transactions
    - transactions_clean

In [2]:
## Create the connection to SQL Lite
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

    return conn

In [3]:
con = create_connection("project (1).db")

In [4]:
cursor = con.cursor()

res = cursor.execute("SELECT type, name FROM sqlite_master WHERE type='table'").fetchall()
all_tables = pd.DataFrame(res)
all_tables.columns = ["type","name"]
all_tables

,type,name
0,table,transactions
1,table,transactions_clean
2,table,monthly_transaction_report
3,table,monthly_agent_report
4,table,monthly_product_report


In [5]:
transactions_raw_to_add = pd.read_sql("SELECT * FROM transactions", con)

In [6]:
transactions_clean = pd.read_sql("SELECT * FROM transactions_clean", con)

## On the job scenario:

The **transactions** table in **project.db** contains transactions from 2019-01-01 to 2020-06-17. This table contains original (raw) data. The data for the period before 2020-01-01 was already cleaned and is stored in the same database, but in the separate table **transactions_clean** (Transaction Date < '2020-01-01').

Imagine that an ETL process uploads new transaction data (the year 2020) every month to the table transactions.




### Task I: Leverage both SQL and Python to extract and clean data using data wrangling techniques

Create the python program that pulls the "new" data from the database and removes unnecessary columns. The program should run monthly. For example:

- On 1st of February 2020, it should take data where Transaction Date is between **'2020-01-01'** and **'2020-01-31'**
- On 1st of March 2020, it should take data where Transaction Date is between **'2020-02-01'** and **'2020-02-29'**


The following columns are unnecessary and should be removed:

1. Columns that are completely empty (should be 4 columns)
2. Constant columns (Policy Level Rate, Is Processed, New To Medicare, Company Business Unit Code)
3. IDs and Names (should be 2 columns)
4. Remove other redundant columns: Carrier Group and Product Code

The final dataframe should have 18 columns.
 
 

## Clean The Raw Transactions File

In [7]:
# Remove the NA columns and Remove Columns that are not needed 
new_transactions_clean_to_add = transactions_raw_to_add.dropna(how='all', axis=1)\
.drop(["Policy Level Rate", "Is Processed", "New To Medicare", "Company Business Unit Code",
       "Insured Name","CarrierCommissionID","Carrier Group","Product Code"], axis=1)

In [8]:
# Check the column names match from the raw transactions dataset and the cleaned dataset
print(new_transactions_clean_to_add.dtypes)
transactions_clean.dtypes

Carrier                             object
Policy #                            object
Effective Date                      object
Transaction Date                    object
Due Date                            object
Premium                            float64
Commissionable Premium             float64
Revenue                            float64
Transaction Code                    object
Policy Year                         object
State                               object
ProductName                         object
Agent Code                          object
Product Type Name                   object
Total Commission                   float64
Insert Date                         object
Date Entered                        object
Carrier Transaction Description     object
dtype: object


Carrier                             object
Policy #                            object
Effective Date                      object
Transaction Date                    object
Due Date                            object
Premium                            float64
Commissionable Premium             float64
Revenue                            float64
Transaction Code                    object
Policy Year                         object
State                               object
ProductName                         object
Agent Code                          object
Product Type Name                   object
Total Commission                   float64
Insert Date                         object
Date Entered                        object
Carrier Transaction Description     object
dtype: object

In [9]:
## Change the date columns in both datasets to be datetime objects

dates_col = ["Effective Date","Transaction Date","Due Date","Date Entered"]

for i in dates_col:
    new_transactions_clean_to_add[i] = pd.to_datetime(new_transactions_clean_to_add[i], format='%Y-%m-%d')
    transactions_clean[i] = pd.to_datetime(transactions_clean[i], format='%Y-%m-%d')

## Check to see if the data types were changed
print(new_transactions_clean_to_add[dates_col].dtypes)
print(transactions_clean[dates_col].dtypes)

Effective Date      datetime64[ns]
Transaction Date    datetime64[ns]
Due Date            datetime64[ns]
Date Entered        datetime64[ns]
dtype: object
Effective Date      datetime64[ns]
Transaction Date    datetime64[ns]
Due Date            datetime64[ns]
Date Entered        datetime64[ns]
dtype: object


In [10]:
## Create a new column in both datasets to find the year-month combination of Transaction Date
# months_in_cleaned = pd.to_datetime(transactions_clean['Transaction Date']).dt.to_period('M').drop_duplicates()
months_in_cleaned = transactions_clean['Transaction Date'].dt.strftime('%Y-%m').drop_duplicates().sort_values()

# Store the last month so I can filter it out of the cleaned transaction dataframe
last_month = months_in_cleaned[-1:]
print(last_month)

# Remove the last month from the filtering list, since it's possible for transactions from the same month in the total transactions dataframe
months_in_cleaned = months_in_cleaned[:-1]
print(months_in_cleaned)

## Creating year-month combination of Transaction Date for full transactions dataframe
new_transactions_clean_to_add['month_year'] = new_transactions_clean_to_add['Transaction Date'].dt.strftime('%Y-%m')
new_transactions_clean_to_add.head(2)

28492    2019-12
Name: Transaction Date, dtype: object
28445    2019-01
28449    2019-02
28451    2019-03
28455    2019-04
28459    2019-05
28465    2019-06
17789    2019-07
28470    2019-08
17788    2019-09
0        2019-10
2        2019-11
Name: Transaction Date, dtype: object


,Carrier,Policy #,Effective Date,Transaction Date,Due Date,Premium,Commissionable Premium,Revenue,Transaction Code,Policy Year,State,ProductName,Agent Code,Product Type Name,Total Commission,Insert Date,Date Entered,Carrier Transaction Description,month_year
0,Union Security Insurance Company,USC0000087,2018-09-15,2019-10-30,2020-01-15,NaN,237.62,16.63,Comm Override,Renewal Business,KY,Mod MS Plan G,USW0000408,Medicare Supplement,16.63,00:39.2,2019-11-13,Commission Override,2019-10
1,Union Security Insurance Company,USC0000087,2018-09-15,2019-10-30,2020-03-15,NaN,237.62,16.63,Comm Override,Renewal Business,KY,Mod MS Plan G,USW0000408,Medicare Supplement,16.63,00:39.2,2019-11-13,Commission Override,2019-10


In [11]:
## Filter out the dates that are already found in the already cleaned transactions dataset
transactions_to_clean_and_append = new_transactions_clean_to_add[~new_transactions_clean_to_add["month_year"].isin(months_in_cleaned)]

## Print the dates not in the already cleaned dataset. This dataset needs to be cleaned and appended to the already cleaned dataframe
transactions_to_clean_and_append["month_year"].unique()

array(['2019-12', '2020-01', '2020-02', '2020-03', '2020-04', '2020-05',
       '2020-06'], dtype=object)

In [12]:
# Remove all entries from the last month of transactions_clean so we can reaggregate for potenial missing data
transactions_clean['month_year'] = transactions_clean['Transaction Date'].dt.strftime('%Y-%m')
transactions_clean = transactions_clean[~transactions_clean['month_year'].isin(last_month)]

# Check to see if the last month is in the list of month years from transaction_clean
print(last_month.isin(list(set(transactions_clean["month_year"]))))

28492    False
Name: Transaction Date, dtype: bool



### Task II: Create a Python script that cleans data and appends it to the appropriate table in the database

Create a python program that checks and cleans the output of Task I and stores it (appends) in the table **transactions_clean**.

- The column that requires our attention is **Transaction Code**. Its values are not consistent, using both upper and lower cases to represent the same thing.

 

In [13]:
## Look at the different values in Transaction Code and edit the Transaction Code column into lowercase

# See the different values in Transaction Code and how many values there are 
print(transactions_to_clean_and_append["Transaction Code"].unique())
print(transactions_to_clean_and_append["Transaction Code"].nunique())

# Change Transaction Code into lowercase and see how many values there are
transactions_to_clean_and_append["Transaction Code"] = new_transactions_clean_to_add["Transaction Code"].str.lower()
print(transactions_to_clean_and_append["Transaction Code"].unique())
print(transactions_to_clean_and_append["Transaction Code"].nunique())

['override' 'Comm Override' 'Chargeback' 'comm override' 'Adjustment'
 'Commission' 'Override' 'Policy Fee' 'ChargeBack' 'adjustment' 'RR']
11
['override' 'comm override' 'chargeback' 'adjustment' 'commission'
 'policy fee' 'rr']
7


<ipython-input-13-3c334db9dc30>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transactions_to_clean_and_append["Transaction Code"] = new_transactions_clean_to_add["Transaction Code"].str.lower()


In [14]:
## Append the newly cleaned transactions to the already cleaned transactions
frames = [transactions_clean, transactions_to_clean_and_append]
transactions_clean_for_reporting = pd.concat(frames)

In [15]:
## Check the number of rows in the already cleaned dataset + new rows = number of rows from total transactions 
print(len(transactions_clean_for_reporting))
len(new_transactions_clean_to_add)

712127


712127

### Task III: Extract data and create a report using Python

We are working on the monthly reports. The reports should be exported in Excel and stored in the database tables. We will be using data in transactions_clean. We should pull the data based on the **Transaction Date**. For example:

- On 1st of April, 2020 it should take data where Transaction Date is between **'2020-03-01'** and **'2020-03-31'**

There are three reports already created in the tables:

- monthly_transaction_report
- monthly_product_report
- monthly_agent_report

We should take the "new" transactions and create a report from them and append to the tables above. The report for each month must have the same columns as SQL tables mentioned above.

This process could obviously be done in SQL procedure as well and it even might be the best option in some cases. However, doing this in Python offers numerous advantages:

- The report can be exported easily to CSV or Excel files
- Python works as a simple connector to any other database running anywhere
- It's very easy to create Python scripts that are scheduled to run on a regular basis.


## Task III.a: Monthly Transaction Report

In [16]:
# Read in monthly transaction report template
monthly_transaction_report = pd.read_sql("SELECT * FROM monthly_transaction_report", con)

# Filter out the last month of the report
monthly_transaction_report = monthly_transaction_report[~monthly_transaction_report['month'].isin(last_month)]
# monthly_transaction_report

In [17]:
## Create Monthly Transaction Report for new months

# Create aggregations
monthly_transaction_report_new_months = transactions_to_clean_and_append.groupby("month_year")\
                                                     .agg({'month_year' : ['count'],
                                                           'Revenue' : ['sum', 'mean'],
                                                           'Premium' : ['sum', 'mean'],
                                                           'Commissionable Premium' : ['sum', 'mean'],
                                                           'Total Commission' : ['sum', 'mean']})

# Reset index so the months don't become row labels
monthly_transaction_report_new_months = monthly_transaction_report_new_months.reset_index()


# # Rename columns to match the template
monthly_transaction_report_new_months.columns = ['month','trx_count',
                                                 "Revenue_sum","Revenue_mean",
                                                 "Premium_sum","Premium_mean",
                                                 "Commissionable Premium_sum", "Commissionable Premium_mean", 
                                                 "Total Commission_sum", "Total Commission_mean"]

In [18]:
# Add new months to template
frames_monthly_txn = [monthly_transaction_report, monthly_transaction_report_new_months]
updated_monthly_txn_report = pd.concat(frames_monthly_txn)
updated_monthly_txn_report.head(3)

,month,trx_count,Revenue_sum,Revenue_mean,Premium_sum,Premium_mean,Commissionable Premium_sum,Commissionable Premium_mean,Total Commission_sum,Total Commission_mean
0,2019-01,215,1712.99,7.967395,422087.77,1963.198930,87.77,21.9425,1712.99,7.967395
1,2019-02,7,48.54,6.934286,10068.99,1438.427143,68.99,34.4950,48.54,6.934286
2,2019-03,18,138.10,7.672222,28214.90,1567.494444,214.90,53.7250,138.10,7.672222


## Task III.b: Monthly Product Report

In [19]:
# Read in monthly product report template
monthly_product_report = pd.read_sql("SELECT * FROM monthly_product_report", con)

# Filter out the last month of the report
monthly_product_report = monthly_product_report[~monthly_product_report['month'].isin(last_month)]

# list(monthly_product_report["month"].drop_duplicates().sort_values())
# monthly_product_report.head(2)

In [20]:
## Create Monthly Product Report for new months

# Create aggregations
monthly_product_report_new_months = transactions_to_clean_and_append.groupby(["ProductName","month_year"])\
                                                     .agg({'month_year' : ['count'],
                                                           'Revenue' : ['sum', 'mean'],
                                                           'Premium' : ['sum', 'mean'],
                                                           'Commissionable Premium' : ['sum', 'mean'],
                                                           'Total Commission' : ['sum', 'mean']})
# Reset index so the months don't become row labels
monthly_product_report_new_months = monthly_product_report_new_months.reset_index()
monthly_product_report_new_months

# Rename columns to match the template
monthly_product_report_new_months.columns = ['ProductName','month','trx_count',
                                                 "Revenue_sum","Revenue_mean",
                                                 "Premium_sum","Premium_mean",
                                                 "Commissionable Premium_sum", "Commissionable Premium_mean", 
                                                 "Total Commission_sum", "Total Commission_mean"]

# monthly_product_report_new_months.head(3)

In [21]:
# Add new months to template
frames_prdt_txn = [monthly_product_report, monthly_product_report_new_months]
updated_prdt_txn_report = pd.concat(frames_prdt_txn).sort_values(["ProductName","month"])

# Check a subset of values
updated_prdt_txn_report[updated_prdt_txn_report["ProductName"] == "DENTAL/VISION"].head(3)

,ProductName,month,trx_count,Revenue_sum,Revenue_mean,Premium_sum,Premium_mean,Commissionable Premium_sum,Commissionable Premium_mean,Total Commission_sum,Total Commission_mean
13,DENTAL/VISION,2019-01,4,8.77,2.1925,87.77,21.9425,87.77,21.9425,8.77,2.1925
14,DENTAL/VISION,2019-02,2,6.89,3.4450,68.99,34.4950,68.99,34.4950,6.89,3.4450
15,DENTAL/VISION,2019-03,4,21.48,5.3700,214.90,53.7250,214.90,53.7250,21.48,5.3700


## Task III.c: Monthly Agent Report

In [22]:
# Read in monthly agent report template
monthly_agent_report = pd.read_sql("SELECT * FROM monthly_agent_report", con)

# Filter out the last month of the report
monthly_agent_report = monthly_agent_report[~monthly_agent_report['month'].isin(last_month)]

# print(list(monthly_agent_report["month"].drop_duplicates().sort_values()))
# monthly_agent_report.sort_values("month").head(5)

In [23]:
## Create Monthly Agent Report for new months

# Create aggregations
monthly_agent_report_new_months = transactions_to_clean_and_append.groupby(["Agent Code","month_year"])\
                                                     .agg({'month_year' : ['count'],
                                                           'Revenue' : ['sum', 'mean'],
                                                           'Premium' : ['sum', 'mean'],
                                                           'Commissionable Premium' : ['sum', 'mean'],
                                                           'Total Commission' : ['sum', 'mean']})
# Reset index so the months don't become row labels
monthly_agent_report_new_months = monthly_agent_report_new_months.reset_index()
monthly_agent_report_new_months

# Rename columns to match the template
monthly_agent_report_new_months.columns = ['Agent Code','month','trx_count',
                                                 "Revenue_sum","Revenue_mean",
                                                 "Premium_sum","Premium_mean",
                                                 "Commissionable Premium_sum", "Commissionable Premium_mean", 
                                                 "Total Commission_sum", "Total Commission_mean"]

# monthly_agent_report_new_months.head(3)

In [24]:
# Add new months to template
frames_agent_txn = [monthly_agent_report, monthly_agent_report_new_months]
updated_agent_txn_report = pd.concat(frames_agent_txn).sort_values(["Agent Code","month"])

# Check a few rows
updated_agent_txn_report[updated_agent_txn_report["Agent Code"] == '3236007'].head(3)

,Agent Code,month,trx_count,Revenue_sum,Revenue_mean,Premium_sum,Premium_mean,Commissionable Premium_sum,Commissionable Premium_mean,Total Commission_sum,Total Commission_mean
2091,3236007,2019-01,5,41.65,8.330000,10000.0,2000.000000,0.0,NaN,41.65,8.330000
2092,3236007,2019-11,28,204.39,7.299643,43200.0,1542.857143,0.0,NaN,204.39,7.299643
9151,3236007,2019-12,24,272.98,11.374167,48000.0,2000.000000,0.0,NaN,272.98,11.374167


### Task IV: Automate reporting by scheduling Python script to run at specified times

Schedule the run of **Task I - III** and simulate run from January to June 2020.

- Schedule the process to run in 2-3 minutes so we can test if it starts automatically
- We simulate the date by using datetime package in Python


```python
# code sample
import datetime
any_day = datetime.date(2020,3,1)
print(any_day)
today = datetime.date.today()
print(today)
```

In this project, we will use the function **datetime.date(2020,3,1)** so we can run the script as it would run in March. In real life, we would replace it with **datetime.date.today()**

We need to start by creating the function return_important_dates that returns the **first** and **last** day of the previous month.


In [25]:
import datetime
any_day = datetime.date(2020,3,1)
print(any_day)
today = datetime.date.today()
print(today)

2020-03-01
2020-12-21


In [26]:
def today_date(first_of_month):
    """
    Create a function that pulls the first and last day of the previous month, and 
    create a date filter for filtering reports
    """
    if first_of_month.month == 1:
        year_interested = first_of_month.year - 1
        month_interested = 12
        day_start = 1
        day_end = calendar.monthrange(year_interested,month_interested)[1]
        
        date_start = datetime.date(year_interested,month_interested,day_start)
        date_end = datetime.date(year_interested,month_interested,day_end)
    else:
        year_interested = first_of_month.year
        month_interested = (first_of_month.month - 1)
        day_start = 1
        day_end = calendar.monthrange(year_interested,month_interested)[1]

        date_start = datetime.date(year_interested,month_interested,day_start)
        date_end = datetime.date(year_interested,month_interested,day_end)
    
    # If the month length is <2, then add a leading 0 to month
    if len(str(date_start.month)) > 1:
        date_filter = str(date_start.year) + '-' + str(date_start.month)
    else:
        date_filter = str(date_start.year) + '-' + '0' + str(date_start.month)
    
    return date_start, date_end, date_filter

In [27]:
# Using function
fn_return = today_date(datetime.date(2020,2,1))
print(fn_return)

(datetime.date(2020, 1, 1), datetime.date(2020, 1, 31), '2020-01')


In [28]:
# Create a list of all the months to run
months_to_run = [datetime.date(2020,1,1),
                 datetime.date(2020,2,1),
                 datetime.date(2020,3,1),
                 datetime.date(2020,4,1),
                 datetime.date(2020,5,1),
                 datetime.date(2020,6,1),
                 datetime.date(2020,7,1)]

In [29]:
def txn_report_generator():
    starting_report_txn = monthly_transaction_report
    for i in (months_to_run):
        fn_return = today_date(i)
        date_filter = fn_return[2]
        
        monthly_txn_report_additional_month = updated_monthly_txn_report[updated_monthly_txn_report["month"] == date_filter]
        month_txn = [starting_report_txn, monthly_txn_report_additional_month]
        starting_report_txn = pd.concat(month_txn).sort_values(["month"])        
        
        print(date_filter)
#     return starting_report.to_csv('mtr.csv')

    starting_report_prod = monthly_product_report
    for i in (months_to_run):
        fn_return = today_date(i)
        date_filter = fn_return[2]
        
        monthly_prod_report_additional_month = updated_prdt_txn_report[updated_prdt_txn_report["month"] == date_filter]
        month_txn = [starting_report_prod, monthly_prod_report_additional_month]
        starting_report_prod = pd.concat(month_txn).sort_values(["month"])        
        
        print(date_filter)
#     return starting_report.to_csv('ptr.csv')

    starting_report_agent = monthly_agent_report
    for i in (months_to_run):
        fn_return = today_date(i)
        date_filter = fn_return[2]
        
        monthly_agent_report_additional_month = updated_agent_txn_report[updated_agent_txn_report["month"] == date_filter]
        month_txn = [starting_report_agent, monthly_agent_report_additional_month]
        starting_report_agent = pd.concat(month_txn).sort_values(["month"])        
        
        print(date_filter)
    return starting_report_txn.to_csv('mtr.csv'), starting_report_prod.to_csv('ptr.csv'), starting_report_agent.to_csv('etr.csv')

In [30]:
schedule.every(15).seconds.do(txn_report_generator)

Every 15 seconds do txn_report_generator() (last run: [never], next run: 2020-12-21 18:42:54)

In [75]:
while True:
    schedule.run_pending()
    time.sleep(1)

2019-12
2020-01
2020-02
2020-03
2020-04
2020-05
2020-06
2019-12
2020-01
2020-02
2020-03
2020-04
2020-05
2020-06
2019-12
2020-01
2020-02
2020-03
2020-04
2020-05
2020-06


KeyboardInterrupt: 

 
### Stretch: Task V

Complete Tasks I, II, III using SQL only. Task IV remains the same in this Stretch Task.
